# Update Drift Monitors Using the Arize GraphQL API

### Step 1: Initialize the GraphQL Client using your developer API Key

In [ ]:
!pip install gql[all]
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport

### Get your API key
First - make sure you have developer permissions. If you are able to visit the [API explorer](https://app.arize.com/graphql), you have developer permissions, if not, please ask your Account Admin to provide you with access. 

The API key can be retrieved from the [API explorer](https://app.arize.com/graphql) page. Click the button on the top right called "Get Your API Key." A modal will pop up with your key, copy that into the `API_KEY` constant below. 

NOTE: this key is different than the SDK key used to send data to Arize. 

In [ ]:
API_KEY = "API_KEY"  # Make sure this is the API key from the GraphQL explorer!

# Select your transport with a defined URL endpoint
transport = RequestsHTTPTransport(
    url="https://app.arize.com/graphql/", headers={"x-api-key": API_KEY}
)

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

### Step 2: Execute a GraphQL query to get all your models and monitors

In [ ]:
# We start this query from your space. Spaces have globally unique IDs. You can get your space ID by visiting app.arize.com.
# The url will be in this format: https://app.arize.com/organizations/:orgId/spaces/:spaceId
# NOTE: this is not the same as the space key used to send data using the SDK
SPACE_ID = "SPACE_ID"

# A re-usable query for fetching your monitors, a page at a time
# Tip: `isManaged` may be set as a boolean parameter on monitors- true for showing only managed monitors
# or false for showing only custom monitors
drift_monitors_query = gql(
    """
  query getDriftMonitors($spaceId: ID!, $cursor: String) {
    space: node(id: $spaceId) {
      ... on Space {
        name
        monitors(
          first: 50, 
          after: $cursor, 
          monitorCategory: drift
        ) {
          pageInfo {
            endCursor
          }
          edges {
            monitor: node {
              id
              name
              threshold
              operator
              currentMetricValue
              status
              isManaged
              primaryMetricWindow {
                dimension {
                  name
                  category
                }
                model {
                  id
                  name
                }
              }
            }
          }
        }
      }
    }
  }
  """
)

# Base query parameters for fetching monitors
params = {"spaceId": SPACE_ID}
# An array of monitors that we will append to
drift_monitors = []
space_name = ""

# Execute the query on the transport. Continue to pull data until there are no more monitors
while True:
    paged_response = client.execute(drift_monitors_query, params)
    space_name = paged_response["space"]["name"]
    # Append the monitors to your list
    drift_monitors.extend(paged_response["space"]["monitors"]["edges"])
    # If there is another page of information, point the cursor to the next page and fetch more
    end_cursor = paged_response["space"]["monitors"]["pageInfo"]["endCursor"]
    print("pageInfo end_cursor %s" % (end_cursor))
    if end_cursor:
        print("There is another page of monitors. Loading more.")
        params["cursor"] = end_cursor
    else:
        # No more monitors to pull. The list is complete!
        break

print("Retrieved {} drift monitors".format(len(drift_monitors)))

#### Step 2a: Print our some of the monitors to check that exports are expected

In [ ]:
import pandas as pd

# The monitors have a nested JSON structure, let's flatten it into a data frame
monitors_df = pd.json_normalize(drift_monitors, sep=".")
monitors_df.head()

### Important: Steps 3-4 involve exporting, manipulating, and importing modified data leveraging google sheets (recommended). If you prefer to use a standard .csv instead, consult the following cell. Otherwise, ignore and proceed to step 3.

In [ ]:
# only consult this section if using csv, then skip ahead to step 5.
# uncomment the following code to create a monitors.csv file in your current directory, modify the csv,
# and convert back to a pandas dataframe

# monitors_csv = monitors_df.to_csv('monitors.csv')
# monitor_updates_df = pd.read_csv('monitors.csv')
# monitor_updates_df.head()

### Step 3: Save monitors in a spreadsheet for editing

In [ ]:
!pip install gspread
# documentation: https://colab.research.google.com/notebooks/snippets/sheets.ipynb
from google.colab import auth

auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()

gc = gspread.authorize(creds)

sheet_name = "{} drift monitors".format(space_name)
sh = gc.create(sheet_name)

# Open our new sheet and add some data
worksheet = gc.open(sheet_name).sheet1

# Write the dataframe to google sheets
worksheet.update([monitors_df.columns.values.tolist()] + monitors_df.values.tolist())

# print the URL
print(sh.url)

### Step 4: Make changes to the spreadsheet and save. Import data from updated spreadsheet
IMPORTANT: Do not add or remove monitors from the spreadsheet - only edit the desired thresholds. If you need to create or delete monitors, consult the documentation for monitor creation and deletion.

In [ ]:
worksheet = gc.open(sheet_name).sheet1

monitor_updates_df = pd.DataFrame(worksheet.get_all_records())
monitor_updates_df.head()

### Step 5: Configure the dataframes

In [ ]:
# set the index of the dataframes to be the ids of the monitors
monitor_updates_df = monitor_updates_df.set_index("monitor.id")
monitors_df = monitors_df.set_index("monitor.id")

### Step 6: Update your monitors the reflect changes by executing the mutation

In [ ]:
import time

update_monitor_mutation = gql(
    """
     mutation updateMonitorThreshold($monitorId: ID!, $threshold: Float!) {
      patchDriftMonitor(input: { monitorId: $monitorId, set: { threshold: $threshold, dynamicAutoThresholdEnabled: false }}) {
        monitor { id, name, threshold }
      }
    }
"""
)

for monitor_id, existing_row in monitor_updates_df.iterrows():
    params = {}
    updated_row = monitor_updates_df.loc[monitor_id].to_dict()
    try:
        original_row = monitors_df.loc[monitor_id].to_dict()
    except:
        print(
            f"❌ Please do not add or remove rows from the spreadsheet, remove monitor {monitor_id}"
        )
        break
    if updated_row["monitor.threshold"] != original_row["monitor.threshold"]:
        params = {
            "monitorId": monitor_id,
            "threshold": updated_row["monitor.threshold"],
        }
        print(f"Update params: {params}")
        result = client.execute(update_monitor_mutation, params)
        print(f"Monitor updated: {result}")
        """
      give a bit of breathing room between updates
    """
        time.sleep(0.1)

print("✅ You have successfully updated your drift monitor(s)")